In [3]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import io
import sys
import logging
from new_strategy import TradingStrategy, Asset, BetSizingMethod, get_bet_sizing
from meta_strategy import MetaLabelingStrategy
import nbimporter
from backtest import Backtest
from add_features import generate_meta_labeled_data, merge_with_raw_features
from metalabel_backtest import MetaModelHandler
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
%load_ext autoreload
%autoreload 2

In [4]:
def split_price_data(price_data: pd.DataFrame, split_ratio: float = 0.7):
    split_idx = int(len(price_data) * split_ratio)
    return price_data.iloc[:split_idx], price_data.iloc[split_idx:]

def run_training_phase(asset: Asset, method: BetSizingMethod, price_data: pd.DataFrame, feature_cols: list):
    # Run base strategy
    past_returns = price_data['close'].pct_change().dropna()
    bet_sizing = get_bet_sizing(method, past_returns)
    
    strategy = TradingStrategy(price_data, asset.value, bet_sizing, method)
    strategy.generate_signals()
    strategy.simulate_trades()
    
    trade_df = strategy.get_trade_data()
    labeled = generate_meta_labeled_data(trade_df)
    #labeled_with_features = merge_with_raw_features(labeled, asset.value)
    labeled_with_features = labeled
    
    print(labeled_with_features.columns)

    output_dir = Path("results_metalabel")
    output_dir.mkdir(exist_ok=True)
    filename = f"train_trades_with_eval_{asset.value}_{method.value}.csv"
    labeled_with_features.to_csv(output_dir / filename, index=False)
    print(f"✅ Training data with rolling eval stats saved: {filename}")

    return labeled_with_features

def train_meta_model(train_df: pd.DataFrame, feature_cols: list) -> MetaModelHandler:
    model = MetaModelHandler()
    model.train(train_df, feature_cols)
    return model

def run_parallel_evaluation(asset, method, test_price_data, signals, meta_model, feature_cols):
    past_returns = test_price_data['close'].pct_change().dropna()
    bet_sizing = get_bet_sizing(method, past_returns)

    #  First, initialize Meta strategy to access feature-enriched data
    meta_strategy = MetaLabelingStrategy(
        test_price_data.copy(), asset.value, bet_sizing, method,
        meta_model_handler=meta_model
    )
    feature_data = meta_strategy.data.copy()

    #  Use feature_data for both strategies
    baseline = TradingStrategy(feature_data.copy(), asset.value, bet_sizing, method)
    baseline.trade_signals = signals

    filtered = MetaLabelingStrategy(feature_data.copy(), asset.value, bet_sizing, method, meta_model_handler=meta_model)
    filtered.trade_signals = signals

    print("\n📊 Baseline features in .data:")
    print(baseline.data.columns.tolist())

    print("\n📊 Filtered features in .data:")
    print(filtered.data.columns.tolist())
    print("\n🕒 Baseline index:", baseline.data.index.name, baseline.data.index.dtype)
    print("🕒 Filtered index:", filtered.data.index.name, filtered.data.index.dtype)

    baseline.simulate_trades()
    filtered.simulate_trades()

    return baseline, filtered


def compare_backtests(baseline: TradingStrategy, filtered: MetaLabelingStrategy, asset: Asset, method: BetSizingMethod):
    print("\n[BASELINE]")
    baseline_bt = Backtest(baseline)
    baseline_bt.run_analysis()
    baseline_buffer = io.StringIO()
    sys.stdout = baseline_buffer
    baseline_bt.print_summary()
    sys.stdout = sys.__stdout__
    baseline_summary = baseline_buffer.getvalue()
    print(baseline_summary)

    print("\n[META-FILTERED]")
    filtered_bt = Backtest(filtered)
    filtered_bt.run_analysis()
    filtered_buffer = io.StringIO()
    sys.stdout = filtered_buffer
    filtered_bt.print_summary()
    sys.stdout = sys.__stdout__
    filtered_summary = filtered_buffer.getvalue()
    print(filtered_summary)

    print(f"\nTrades Rejected by Meta-Model: {filtered.rejected_trades}")

    output_dir = "results_metalabel"
    os.makedirs(output_dir, exist_ok=True)

    asset_name = asset.value.lower()
    method_name = method.value.lower()

    baseline_csv = f"baseline_{asset_name}_{method_name}.csv"
    filtered_csv = f"filtered_{asset_name}_{method_name}.csv"
    baseline_txt = f"baseline_{asset_name}_{method_name}.txt"
    filtered_txt = f"filtered_{asset_name}_{method_name}.txt"

    baseline.get_trade_data().to_csv(os.path.join(output_dir, baseline_csv), index=False)
    filtered.get_trade_data().to_csv(os.path.join(output_dir, filtered_csv), index=False)

    with open(os.path.join(output_dir, baseline_txt), "w") as f:
        f.write(baseline_summary)

    with open(os.path.join(output_dir, filtered_txt), "w") as f:
        f.write(filtered_summary)

    print(f"\n✅ CSVs saved to: {output_dir}")
    print(f" - {baseline_csv}")
    print(f" - {filtered_csv}")
    print(f"\n✅ Summaries saved to:")
    print(f" - {baseline_txt}")
    print(f" - {filtered_txt}")


if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)

    asset = Asset.XAUUSD
    method = BetSizingMethod.FIXED

    feature_cols = ["attempt", "ma_14", "min_price_30", "max_price_30","atr_14"]
    price_path = Path(f"data/processed/{asset.value}/combined_data.csv")
    price_data = pd.read_csv(price_path, index_col='timestamp', parse_dates=True)

    # Split
    train_data, test_data = split_price_data(price_data)

    # Training
    train_trades = run_training_phase(asset, method, train_data, feature_cols)
    train_trades['session_code'] = train_trades['session'].map({
    'asian': 0,
    'london': 1,
    'us': 2
    })


    clean_features = train_trades[feature_cols].replace([np.inf, -np.inf], np.nan).dropna()
    train_trades_cleaned = train_trades.loc[clean_features.index]
    meta_model = train_meta_model(train_trades_cleaned, feature_cols)

    # Reuse signals
    signal_gen = TradingStrategy(test_data.copy(), asset.value, get_bet_sizing(method), method)
    signal_gen.generate_signals()
    shared_signals = signal_gen.trade_signals

    print("\n🔍 Entry Time Check (from shared_signals dict):")
    print("Type:", type(shared_signals))
    print("Keys in shared_signals:", list(shared_signals.keys()))

    # Try to peek at the first signal (depending on structure)
    first_key = next(iter(shared_signals))
    first_signal_list = shared_signals[first_key]
    first_signal = first_signal_list[0]  # Get the first signal from the list
    print("Type of entry_time:", type(first_signal['entry_time']))

    # Evaluation
    base_strat, filtered_strat = run_parallel_evaluation(
        asset, method, test_data, shared_signals, meta_model, feature_cols
    )

    # Compare
    compare_backtests(base_strat, filtered_strat, asset, method)



2025-06-10 11:48:59,648 - INFO - Strategy initialized for XAUUSD using FixedFractionalBetSizing
2025-06-10 11:53:30,049 - INFO - Strategy initialized for XAUUSD using FixedFractionalBetSizing
2025-06-10 11:54:13,526 - INFO - Strategy initialized for XAUUSD using FixedFractionalBetSizing
2025-06-10 11:54:13,755 - INFO - Strategy initialized for XAUUSD using FixedFractionalBetSizing
2025-06-10 11:54:13,813 - INFO - Strategy initialized for XAUUSD using FixedFractionalBetSizing
2025-06-10 11:54:35,219 - INFO - Exported detailed trades to data/results/trades_detailed_XAUUSD_fixed.csv
2025-06-10 11:54:35,283 - INFO - Saved backtest results to data/results/backtest_results_XAUUSD_fixed.txt
2025-06-10 11:54:35,548 - INFO - Exported detailed trades to data/results/trades_detailed_XAUUSD_fixed.csv
2025-06-10 11:54:35,606 - INFO - Saved backtest results to data/results/backtest_results_XAUUSD_fixed.txt
